<a href="https://colab.research.google.com/github/JI411/coal-composition-control/blob/dev/src/InstanceSegmentation/(CompTech_28_01)_TorchVision_Instance_Segmentation_Mask_R_CNN_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TorchVision Instance Segmentation Mask R-CNN Finetuning

sources: [Torchvision Tutorial](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/torchvision_finetuning_instance_segmentation.ipynb?authuser=1#scrollTo=cFHKCvCTxiff), [Annotation convertion](https://towardsdatascience.com/generating-image-segmentation-masks-the-easy-way-dd4d3656dbd1)

In [ ]:
!nvidia-smi

In [ ]:
%%capture
!pip install gdown
!gdown --id 1YrJAEBJDDzqkdMnCno0RzJlUbQjJ6QGT  # https://drive.google.com/file/d/1YrJAEBJDDzqkdMnCno0RzJlUbQjJ6QGT/view
!unzip example.zip
!git clone https://github.com/JI411/coal-composition-control -b dev
%cd coal-composition-control
!pip install -r requirements.txt
%cd /content/coal-composition-control/src/InstanceSegmentation

In [ ]:
!pip install -q cython
!pip install -qU 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install -qU numpy
# !pip install -q albumentations==0.4.6

## Defining the Dataset

In [ ]:
import torch
import numpy as np
from PIL import Image
from pathlib import Path
from dataset import CoalFractionDataset

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

That's it, this will make model be ready to be trained and evaluated on our custom dataset.

## Training and evaluation functions

In `references/detection/,` we have a number of helper functions to simplify training and evaluating detection models.
Here, we will use `references/detection/engine.py`, `references/detection/utils.py` and `references/detection/transforms.py`.

Let's copy those files (and their dependencies) in here so that they are available in the notebook

In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
# git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../



Let's write some helper functions for data augmentation / transformation, which leverages the functions in `refereces/detection` that we have just copied:


In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(p=0.5))
        transforms.append(T.RandomIoUCrop())
        transforms.append(T.RandomZoomOut(p=0.2))
        transforms.append(T.RandomPhotometricDistort(p=0.2))

    return T.Compose(transforms)

#### Testing forward() method 

Before iterating over the dataset, it’s good to see what the model expects during training and inference time on sample data.


In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

dataset = CoalFractionDataset(
    root='/content/few_data/', 
    vgg_json='/content/few_data.json', 
    width=1320,
    height=512,
    transforms=get_transform(train=True)
    )
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn
)
# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

#### Note that we do not need to add a mean/std normalization nor image rescaling in the data transforms, as those are handled internally by the Mask R-CNN model.

### Putting everything together

We now have the dataset class, the models and the data transforms. Let's instantiate them

In [ ]:
# use our dataset and defined transformations
dataset = dataset = CoalFractionDataset(
    root='/content/few_data/', 
    vgg_json='/content/few_data.json', 
    width=1320,
    height=512,
    transforms=get_transform(train=True)
    )
dataset_test = dataset = CoalFractionDataset(
    root='/content/few_data/', 
    vgg_json='/content/few_data.json', 
    width=1320,
    height=512,
    transforms=get_transform(train=False)
    )

# split the dataset in train and test set
torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

Now let's instantiate the model and the optimizer

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

And now let's train the model for 10 epochs, evaluating at the end of every epoch.

In [ ]:
# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

In [ ]:
torch.save(model, '/content/mask-rcnn.pth')

In [ ]:
# model = torch.load('/content/mask-rcnn.pth')
model.eval()
# model.roi_heads.detections_per_img = 100
model.roi_heads.box_score_thresh=0.5
model.roi_heads.box_nms_thresh=0.3

Now that training has finished, let's have a look at what it actually predicts in a test image

In [ ]:
# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

Printing the prediction shows that we have a list of dictionaries. Each element of the list corresponds to a different image. As we have a single image, there is a single dictionary in the list.
The dictionary contains the predictions for the image we passed. In this case, we can see that it contains `boxes`, `labels`, `masks` and `scores` as fields.

In [ ]:
# prediction

Let's inspect the image and the predicted segmentation masks.

For that, we need to convert the image, which has been rescaled to 0-1 and had the channels flipped so that we have it in `[C, H, W]` format.

In [ ]:
prediction[0]['boxes'].shape

In [ ]:
prediction[0]['masks'].shape

In [ ]:
import torchvision.transforms as transforms
from torchvision.utils import draw_segmentation_masks, draw_bounding_boxes

to_pil_image = transforms.ToPILImage()
byte_img = img.mul(255).byte()

In [ ]:
def mask_on_image(byte_img, prediction, th=0.7):
  masks = torch.squeeze(prediction[0]['masks']).cpu()
  threshold = torch.Tensor([th])
  masks = (masks > threshold).bool()
  masks_on_image = draw_segmentation_masks(
      image = byte_img,
      masks = masks,
      alpha=0.6
      )
  return to_pil_image(masks_on_image)

In [ ]:
def box_on_image(byte_img, prediction):
  boxes = torch.squeeze(prediction[0]['boxes'])
  boxes_on_image = draw_bounding_boxes(
      image=byte_img,
      boxes=boxes,
      width=4
      )
  return to_pil_image(boxes_on_image)

In [ ]:
to_pil_image(byte_img)

In [ ]:
mask_on_image(byte_img, prediction, th=0.9)

In [ ]:
mask_on_image(byte_img, prediction, th=0.8)

In [ ]:
mask_on_image(byte_img, prediction, th=0.7)

In [ ]:
mask_on_image(byte_img, prediction, th=0.6)

In [ ]:
mask_on_image(byte_img, prediction, th=0.5)

In [ ]:
mask_on_image(byte_img, prediction, th=0.4)

In [ ]:
mask_on_image(byte_img, prediction, th=0.3)

In [ ]:
mask_on_image(byte_img, prediction, th=0.2)

In [ ]:
mask_on_image(byte_img, prediction, th=0.1)

In [ ]:
box_on_image(byte_img, prediction)